In [1]:
%load_ext autoreload
%autoreload 2

from kloppy import sportec
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mplsoccer import Pitch
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm
pd.set_option('display.max_columns', None)  # 모든 열 출력
from unravel.soccer import KloppyPolarsDataset

import os 
import pickle
import sys

sys.path.append(os.path.abspath(".."))
from pressing_intensity import CustomPressingIntensity
from visualization import plot_single_frame_positions, plot_window_frame_positions
from config import TEAMNAME2ID, TEAMID2NAME

coordinates = "secondspectrum"

2025-06-11 16:24:41.931871: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 16:24:41.934279: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 16:24:41.961076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-11 16:24:41.961107: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-11 16:24:41.961131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [234]:
data_path = "/data/MHL/bepro/processed/"
results = []

if os.path.exists(data_path):
    match_id_lst = os.listdir(data_path)
    total_dict = {match_id : {} for match_id in match_id_lst}
    for match_id in match_id_lst:
        print(f"Load {match_id}")
        with open(f"{data_path}/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
            match_dict = pickle.load(f)
        tracking_df = match_dict['tracking_df']
        event_df = match_dict['event_df']
        meta_data = match_dict['meta_data']
        teams_dict = match_dict['teams']
        with open(f"{data_path}/{match_id}/{match_id}_presing_intensity.pkl", "rb") as f:
            pressing_df = pickle.load(f)

        total_df = pd.merge(tracking_df, pressing_df, on=['game_id', 'period_id', 'timestamp', 'frame_id'], how='left')
        # total_df = _normalize_coordinate_direction(total_df, teams_dict['Home']['pID'].iloc[0])
        total_dict[match_id] = {}
        total_dict[match_id]['tracking_df'] = total_df
        total_dict[match_id]['event_df'] = event_df
        total_dict[match_id]['meta_data'] = meta_data
        total_dict[match_id]['Home'] = match_dict['teams']['Home']
        total_dict[match_id]['Away'] = match_dict['teams']['Away']


Load 126293
Load 126298
Load 126306
Load 126309
Load 126315
Load 126319
Load 126325
Load 126332
Load 126341
Load 126348
Load 126350
Load 126356
Load 126364
Load 126367
Load 126378
Load 126380
Load 126386
Load 126391
Load 126401
Load 126408
Load 126411
Load 126418
Load 126424
Load 126429
Load 126433
Load 126444
Load 126448
Load 126455
Load 126458
Load 126466
Load 126473
Load 126476
Load 153364
Load 153373
Load 153379
Load 153381
Load 153385
Load 153387
Load 153390
Load 126285


In [254]:
match_id = "126315"

total_df = total_dict[match_id]['tracking_df'].copy()
event_df = total_dict[match_id]['event_df'].copy()
meta_data = total_dict[match_id]['meta_data'].copy()
home_team = total_dict[match_id]['Home'].copy()
away_team = total_dict[match_id]['Away'].copy()
teams_dict = {'Home' : home_team, 'Away' : away_team}

In [255]:
filtered_event_df = pd.read_csv(f"{data_path}/{match_id}/valid_events_filtered.csv")
filtered_event_df.head()

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy
0,126315,0.0,0,1,2.167,3.967,4648,360389,62365.0,-1,-1,pass,4,Successful,4.0,-1.0,52.5840,35.8428,69.0480,46.8860,69.0480,46.8860,16.4640,11.0432
1,126315,5.0,2,1,4.633,6.000,4648,62365,500133.0,-1,-1,pass,4,Successful,6.0,-1.0,68.9430,47.7428,69.8565,31.0964,69.8565,31.0964,0.9135,-16.6464
2,126315,NaN,4,1,6.001,NaN,4648,500133,NaN,-1,-1,dribble,2,NaN,NaN,NaN,69.8565,31.0964,57.4665,29.8928,NaN,NaN,-12.3900,-1.2036
3,126315,8.0,5,1,10.667,12.433,4648,500133,145701.0,-1,-1,pass,4,Successful,9.0,-1.0,57.4665,29.8928,59.6820,15.9664,59.6820,15.9664,2.2155,-13.9264
4,126315,9.0,7,1,12.433,13.533,4648,145701,77414.0,-1,-1,pass,4,Successful,11.0,-1.0,59.6820,15.9664,41.8845,14.3276,41.8845,14.3276,-17.7975,-1.6388


In [256]:
filtered_event_df['time_seconds'] = (filtered_event_df['time_seconds'] / 0.04).round() * 0.04
filtered_event_df['relative_time_seconds'] = (filtered_event_df['relative_time_seconds'] / 0.04).round() * 0.04
filtered_event_df['time_seconds'] = pd.to_timedelta(filtered_event_df['time_seconds'], unit='s')
filtered_event_df['relative_time_seconds'] = pd.to_timedelta(filtered_event_df['relative_time_seconds'], unit='s')

filtered_event_df['player_id'] = filtered_event_df['player_id'].astype(str)

In [257]:
teams_df = pd.concat([home_team, away_team])
teams_df.reset_index(drop=True, inplace=True)
teams_df['player_code'] = teams_df.apply(lambda row : row['team'][0] + str(row['xID']).zfill(2), axis=1)
teams_df.head()

,player,position,team,jID,pID,tID,xID,player_code
0,Joonhong Kim,GK,Home,17,356626,2353,0,H00
1,Seunguk Park,CB,Home,25,500576,2353,1,H01
2,Bongsoo Kim,CB,Home,15,500569,2353,2,H02
3,Suil Park,RB,Home,23,500562,2353,3,H03
4,Mingyu Park,LB,Home,88,356609,2353,4,H04


In [258]:
filtered_event_df = filtered_event_df.merge(
    teams_df,
    how='left',
    left_on='player_id',
    right_on='pID'
)
filtered_event_df.head()

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy,player,position,team,jID,pID,tID,xID,player_code
0,126315,0.0,0,1,0 days 00:00:02.160000,0 days 00:00:03.960000,4648,360389,62365.0,-1,-1,pass,4,Successful,4.0,-1.0,52.5840,35.8428,69.0480,46.8860,69.0480,46.8860,16.4640,11.0432,Kang Hyun Lee,CM,Away,24,360389,4648,6,A06
1,126315,5.0,2,1,0 days 00:00:04.640000,0 days 00:00:06,4648,62365,500133.0,-1,-1,pass,4,Successful,6.0,-1.0,68.9430,47.7428,69.8565,31.0964,69.8565,31.0964,0.9135,-16.6464,Youngkyu Ahn,CB,Away,6,62365,4648,3,A03
2,126315,NaN,4,1,0 days 00:00:06,NaT,4648,500133,NaN,-1,-1,dribble,2,NaN,NaN,NaN,69.8565,31.0964,57.4665,29.8928,NaN,NaN,-12.3900,-1.2036,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
3,126315,8.0,5,1,0 days 00:00:10.680000,0 days 00:00:12.440000,4648,500133,145701.0,-1,-1,pass,4,Successful,9.0,-1.0,57.4665,29.8928,59.6820,15.9664,59.6820,15.9664,2.2155,-13.9264,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
4,126315,9.0,7,1,0 days 00:00:12.440000,0 days 00:00:13.520000,4648,145701,77414.0,-1,-1,pass,4,Successful,11.0,-1.0,59.6820,15.9664,41.8845,14.3276,41.8845,14.3276,-17.7975,-1.6388,Mingi Lee,LB,Away,3,145701,4648,1,A01


In [259]:
total_df.loc[total_df['period_id'] == 2, 'timestamp'] -= pd.Timedelta(minutes=45)
total_df = total_df.merge(
    teams_df[['pID', 'player_code']],
    how='left',
    left_on = 'id',
    right_on = 'pID'
)
total_df.drop(['pID'], axis=1, inplace=True)

# Ball id
nan_mask = pd.isna(total_df['player_code'])
if total_df.loc[nan_mask, 'id'].nunique() != 1:
    print('error') 
else:
    if total_df.loc[nan_mask, 'id'].unique()[0] != 'ball':
        print('error')
total_df.loc[nan_mask, 'player_code'] = total_df.loc[nan_mask, 'id']
total_df[pd.isna(total_df['player_code'])]


,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,player_code


In [260]:
wide_tracking_df = total_df.pivot_table(
    index= ['period_id', 'timestamp'],
    columns='player_code',
    values=['x', 'y', 'vx', 'vy', 'v', 'ax', 'ay', 'a']
)

wide_tracking_df.columns = [f'{player_code}_{value}' for value, player_code in wide_tracking_df.columns]
wide_tracking_df.reset_index(inplace=True)
wide_tracking_df.head()

,period_id,timestamp,A00_a,A01_a,A02_a,A03_a,A04_a,A05_a,A06_a,A07_a,A08_a,A09_a,A10_a,A13_a,A14_a,A15_a,A17_a,A18_a,H00_a,H01_a,H02_a,H03_a,H04_a,H05_a,H06_a,H07_a,H08_a,H09_a,H10_a,H12_a,H15_a,H17_a,H18_a,H19_a,ball_a,A00_ax,A01_ax,A02_ax,A03_ax,A04_ax,A05_ax,A06_ax,A07_ax,A08_ax,A09_ax,A10_ax,A13_ax,A14_ax,A15_ax,A17_ax,A18_ax,H00_ax,H01_ax,H02_ax,H03_ax,H04_ax,H05_ax,H06_ax,H07_ax,H08_ax,H09_ax,H10_ax,H12_ax,H15_ax,H17_ax,H18_ax,H19_ax,ball_ax,A00_ay,A01_ay,A02_ay,A03_ay,A04_ay,A05_ay,A06_ay,A07_ay,A08_ay,A09_ay,A10_ay,A13_ay,A14_ay,A15_ay,A17_ay,A18_ay,H00_ay,H01_ay,H02_ay,H03_ay,H04_ay,H05_ay,H06_ay,H07_ay,H08_ay,H09_ay,H10_ay,H12_ay,H15_ay,H17_ay,H18_ay,H19_ay,ball_ay,A00_v,A01_v,A02_v,A03_v,A04_v,A05_v,A06_v,A07_v,A08_v,A09_v,A10_v,A13_v,A14_v,A15_v,A17_v,A18_v,H00_v,H01_v,H02_v,H03_v,H04_v,H05_v,H06_v,H07_v,H08_v,H09_v,H10_v,H12_v,H15_v,H17_v,H18_v,H19_v,ball_v,A00_vx,A01_vx,A02_vx,A03_vx,A04_vx,A05_vx,A06_vx,A07_vx,A08_vx,A09_vx,A10_vx,A13_vx,A14_vx,A15_vx,A17_vx,A18_vx,H00_vx,H01_vx,H02_vx,H03_vx,H04_vx,H05_vx,H06_vx,H07_vx,H08_vx,H09_vx,H10_vx,H12_vx,H15_vx,H17_vx,H18_vx,H19_vx,ball_vx,A00_vy,A01_vy,A02_vy,A03_vy,A04_vy,A05_vy,A06_vy,A07_vy,A08_vy,A09_vy,A10_vy,A13_vy,A14_vy,A15_vy,A17_vy,A18_vy,H00_vy,H01_vy,H02_vy,H03_vy,H04_vy,H05_vy,H06_vy,H07_vy,H08_vy,H09_vy,H10_vy,H12_vy,H15_vy,H17_vy,H18_vy,H19_vy,ball_vy,A00_x,A01_x,A02_x,A03_x,A04_x,A05_x,A06_x,A07_x,A08_x,A09_x,A10_x,A13_x,A14_x,A15_x,A17_x,A18_x,H00_x,H01_x,H02_x,H03_x,H04_x,H05_x,H06_x,H07_x,H08_x,H09_x,H10_x,H12_x,H15_x,H17_x,H18_x,H19_x,ball_x,A00_y,A01_y,A02_y,A03_y,A04_y,A05_y,A06_y,A07_y,A08_y,A09_y,A10_y,A13_y,A14_y,A15_y,A17_y,A18_y,H00_y,H01_y,H02_y,H03_y,H04_y,H05_y,H06_y,H07_y,H08_y,H09_y,H10_y,H12_y,H15_y,H17_y,H18_y,H19_y,ball_y
0,1.0,0 days 00:00:02.200000,0.157415,0.344774,0.440375,2.115579,3.931717,1.500903,1.858172,3.234928,2.313510,0.432555,2.093813,NaN,NaN,NaN,NaN,NaN,0.636341,0.684999,0.958624,1.896696,0.716976,1.540083,0.794225,0.476901,1.184674,6.000000,4.519603,NaN,NaN,NaN,NaN,NaN,0.000000,-0.009143,0.095604,0.126098,-0.672857,-1.736133,-0.236605,-0.789188,-3.209864,-2.288382,-0.000496,-1.959916,NaN,NaN,NaN,NaN,NaN,0.595879,0.644504,0.889314,-1.886819,0.269950,1.301225,0.789385,0.277449,-1.174694,0.088101,4.508509,NaN,NaN,NaN,NaN,NaN,0.000000,0.157149,0.331254,-0.421935,2.005727,3.527639,1.482136,1.682256,0.401910,-0.340050,0.432555,-0.736736,NaN,NaN,NaN,NaN,NaN,-0.223288,-0.232032,-0.357883,0.193307,0.664216,0.823813,0.087543,0.387888,0.153449,-6.456073,-0.316477,NaN,NaN,NaN,NaN,NaN,0.000000,0.010506,0.017269,0.360554,0.058589,0.825167,0.665585,1.024224,0.675646,0.546679,0.150971,1.062552,NaN,NaN,NaN,NaN,NaN,1.007542,0.525383,0.580578,0.288106,0.469734,0.930481,0.856644,0.985753,0.412375,1.061619,1.852147,NaN,NaN,NaN,NaN,NaN,1.874694,-0.000366,0.010984,0.320033,0.036300,-0.652317,-0.319823,0.487285,-0.671020,-0.460582,0.016409,-0.789224,NaN,NaN,NaN,NaN,NaN,1.007020,0.509849,0.562702,0.259527,-0.457413,0.881450,0.828473,0.693277,-0.365545,0.946555,1.816391,NaN,NaN,NaN,NaN,NaN,1.556758,-0.010500,0.013325,-0.166067,0.045988,0.505356,-0.583709,-0.900882,-0.078924,0.294486,0.150077,-0.711437,NaN,NaN,NaN,NaN,NaN,-0.032452,-0.126812,-0.142961,0.125103,0.106878,0.298062,-0.217880,-0.700768,0.190865,-0.480697,-0.362178,NaN,NaN,NaN,NaN,NaN,1.044501,49.496500,10.665100,15.646100,16.524300,12.002500,0.037200,-0.347900,0.215800,0.190100,0.465900,0.278700,NaN,NaN,NaN,NaN,NaN,-46.617500,-20.065300,-18.285100,-16.425300,-16.240800,-8.407800,-8.816900,-6.780500,-3.982700,0.013100,0.439800,NaN,NaN,NaN,NaN,NaN,0.386200,-1.8148,-15.734300,-5.891400,6.277600,23.852000,17.127400,0.497700,9.290100,-24.561000,-18.951000,-15.385400,NaN,NaN,NaN,NaN,NaN,0.418800,-11.041000,4.463700,-22.461800,16.264900,-16.614100,-1.638100,5.710600,-22.139300,21.514900,10.260900,NaN,NaN,NaN,NaN,NaN,2.048200
1,1.0,0 days 00:00:02.240000,0.113183,0.447191,0.613509,2.165875,4.670158,1.768981,2.330601,3.725155,2.653523,0.496971,2.373170,NaN,NaN,NaN,NaN,NaN,0.694116,0.769071,1.

In [262]:
total_df = total_dict[match_id]['tracking_df'].copy()

# ball carrier에 대해 pressing intensity가 0.7보다 큰 경우 pressed_df 구성
pressed_dict = {}
ball_carrier_df = total_df[total_df['is_ball_carrier'] == True].copy()
ball_carrier_df = ball_carrier_df[ball_carrier_df['ball_state']=='alive']
ball_carrier_df.sort_values(['period_id', 'frame_id'], inplace=True)

for idx, row in tqdm(ball_carrier_df.iterrows(), desc= "Get Pressing Intensity", miniters=len(ball_carrier_df)//10):
    if len(np.where(row['rows'] == row['id'])[0]) != 0:
        pressed_axis = 'rows'
        presser_axis = 'columns'
        id_loc = np.where(row[pressed_axis] == row['id'])[0]
        # 다중 list nested 구조로 되어 있을 수 있으므로 tolist()를 두 번 적용
        pressing_values = row['probability_to_intercept'][id_loc].tolist()[0].tolist()
    elif len(np.where(row['columns'] == row['id'])[0]) != 0:
        pressed_axis = 'columns'
        presser_axis = 'rows'
        id_loc = np.where(row[pressed_axis] == row['id'])[0]
        pressing_values = [x[id_loc] for x in row['probability_to_intercept']]
    else:
        continue
    if max(pressing_values) > 0.9:
        pressed_dict[idx] = {}
        pressed_dict[idx]['pressing_value'] = max(pressing_values)
        max_idx = pressing_values.index(max(pressing_values))
        pressed_dict[idx]['pressing_player'] = row[presser_axis][max_idx]

pressed_df = ball_carrier_df.loc[list(pressed_dict.keys())].copy()
pressed_df['pressing_values'] = [d['pressing_value'] for d in pressed_dict.values()]
pressed_df['pressing_player'] = [d.get('pressing_player') for d in pressed_dict.values()]        

period_list = []
for period_id in pressed_df['period_id'].unique():
    period_df = pressed_df[pressed_df['period_id']==period_id].copy()
    # frame_id 차이가 50 프레임 이상인 경우 새로운 시퀀스로 판단 (즉, 연속된 pressed 행이 아닐 경우)
    period_df['frame_diff'] = period_df['frame_id'].diff()
    period_df['sequence_id'] = (period_df['frame_diff'] > 125).cumsum()

    # 각 시퀀스별 첫 번째 frame을 기준으로 X와 Y를 설정하기 위해 첫번째 frame 정보를 구함
    first_frames = period_df.groupby('sequence_id', as_index=False)[['timestamp', 'period_id', 'frame_id', 'id', 'team_id', 'pressing_player']].first()

    # total_df 에서 필요한 컬럼만 추출
    lookup = total_df[['period_id', 'frame_id', 'id', 'v']]

    # pressing_player 컬럼은 total_df.id 와 매칭되어 있으므로,
    # 컬럼명을 맞춰서 merge
    first_frames = first_frames.merge(
        lookup.rename(columns={'id': 'pressing_player'}),
        on=['period_id', 'frame_id', 'pressing_player'],
        how='left'
    )

    # 압박하는 선수의 속도가 2.0 m/s 인 경우만 압박으로 간주
    first_frames = first_frames[first_frames['v'] >= 2.0]
    period_list.append(first_frames)

first_frames_df = pd.concat(period_list, axis=0, ignore_index=True)


Get Pressing Intensity: 85429it [00:03, 21411.49it/s]


In [263]:
filtered_event_df['type_name'].unique()

array(['pass', 'dribble', 'recovery', 'interception', 'foul',
       'freekick_crossed', 'cross', 'keeper_punch', 'throw_in', 'take_on',
       'corner_crossed', 'shot', 'goalkick', 'block', 'clearance',
       'shot_penalty', 'keeper_save'], dtype=object)

In [264]:
possession_gained_events = ['pass', 'dribble', 'recovery', 'interception', 'cross', 'foul', 'throw_in', 'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick', 'clearance'] 

for ev in possession_gained_events:
    print(f"{ev} | {filtered_event_df[filtered_event_df['type_name']==ev]['result_name'].unique()}")

pass | ['Successful' 'Unsuccessful']
dribble | [nan]
recovery | [nan]
interception | [nan]
cross | ['Unsuccessful' 'Successful']
foul | [nan]
throw_in | ['Successful' 'Unsuccessful']
take_on | ['Unsuccessful' 'Successful']
shot | ['Off Target' 'Blocked' 'On Target' 'Goal']
freekick_crossed | ['Successful' 'Unsuccessful']
corner_crossed | ['Successful' 'Unsuccessful']
goalkick | ['Unsuccessful' 'Successful']
clearance | [nan]


In [265]:
def check_pressing_success(row, event_df, teams_dict):
    possession_gained_events = ['pass', 'dribble', 'recovery', 'interception', 'cross', 'throw_in', 'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick'] 
    
    if row['team_id'] == teams_dict['Home']['tID'].unique()[0]:
        pressing_team = teams_dict['Away']['tID'].unique()[0]

    elif row['team_id'] == teams_dict['Away']['tID'].unique()[0]:
        pressing_team = teams_dict['Home']['tID'].unique()[0]

    check_timegap = pd.Timedelta(seconds=5)
    window_events = event_df[
        (event_df['period_id'] == row['period_id']) &
        (event_df['time_seconds'] >= row['timestamp']) &
        (event_df['time_seconds'] <= row['timestamp'] + check_timegap)
        ]
    # event_teams = window_events['team_name'].unique()
    event_teams = window_events['tID'].unique()
    # event_team_ids = [TEAMNAME2ID[x] for x in event_teams]

    # if pressing_team in event_team_ids:
    if pressing_team in event_teams:
        # pressing_team_events = window_events[window_events['team_name'] == TEAMID2NAME[pressing_team]]
        pressing_team_events = window_events[window_events['tID'] == pressing_team]
        for _, row in pressing_team_events.iterrows():
            if row['type_name'] in possession_gained_events:
                if row['type_name'] in ['dribble', 'recovery', 'interception', 'cross', 'throw_in', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick']:
                    return True
                else: # [Pass, take_on]
                    result = row['result_name']
                    if result == 'Successful':
                        return True
        return False
    else:
        return False

first_frames_df['ball_ownership_changed'] = first_frames_df.apply(check_pressing_success, axis=1, event_df=filtered_event_df, teams_dict=teams_dict) # window=150은 예시

In [289]:
first_frames_df[(first_frames_df['period_id'] == 1) & (first_frames_df['frame_id']==58314)]

,sequence_id,timestamp,period_id,frame_id,id,team_id,pressing_player,v,ball_ownership_changed
88,88,0 days 00:38:52.560000,1.0,58314,356624,2353,250101,2.612496,True


In [292]:
from IPython.display import display

row = first_frames_df[(first_frames_df['period_id'] == 1) & (first_frames_df['frame_id']==58314)].iloc[0]

possession_gained_events = ['pass', 'dribble', 'recovery', 'interception', 'cross', 'throw_in', 'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick'] 

if row['team_id'] == teams_dict['Home']['tID'].unique()[0]:
    pressing_team = teams_dict['Away']['tID'].unique()[0]

elif row['team_id'] == teams_dict['Away']['tID'].unique()[0]:
    pressing_team = teams_dict['Home']['tID'].unique()[0]

print(f"Pressing Team : {pressing_team}")
print(f"Pressing Time : {row['timestamp']}")
print(f"Results : {row['ball_ownership_changed']}")

check_timegap = pd.Timedelta(seconds=5)

prev_window_events = filtered_event_df[
        (filtered_event_df['period_id'] == row['period_id']) &
        (filtered_event_df['time_seconds'] >= row['timestamp'] - pd.Timedelta(seconds=5)) &
        (filtered_event_df['time_seconds'] <= row['timestamp'])
        ].tail(5)

aft_window_events = filtered_event_df[
        (filtered_event_df['period_id'] == row['period_id']) &
        (filtered_event_df['time_seconds'] >= row['timestamp']) &
        (filtered_event_df['time_seconds'] <= row['timestamp'] + check_timegap)
        ]
# event_teams = window_events['team_name'].unique()
event_teams = aft_window_events['tID'].unique()
# event_team_ids = [TEAMNAME2ID[x] for x in event_teams]

print("Before Pressing")
display(prev_window_events[['period_id', 'time_seconds', 'team_id', 'player_id', 'type_name', 'result_name']])

print("After Pressing")
display(aft_window_events[['period_id', 'time_seconds', 'team_id','player_id', 'type_name', 'result_name']])

Pressing Team : 4648
Pressing Time : 0 days 00:38:52.560000
Results : True
Before Pressing


,period_id,time_seconds,team_id,player_id,type_name,result_name
743,1,0 days 00:38:49.840000,2353,500562,throw_in,Successful
744,1,0 days 00:38:51.720000,2353,500583,pass,Successful
745,1,0 days 00:38:52.560000,2353,356624,dribble,NaN


After Pressing


,period_id,time_seconds,team_id,player_id,type_name,result_name
745,1,0 days 00:38:52.560000,2353,356624,dribble,NaN
746,1,0 days 00:38:53.200000,4648,250102,recovery,NaN
747,1,0 days 00:38:53.200000,4648,250102,pass,Successful
748,1,0 days 00:38:55,4648,250101,dribble,NaN
749,1,0 days 00:38:55.880000,4648,250101,foul,NaN


In [293]:
total_df[
    (total_df['period_id'] == row['period_id']) &
    (total_df['timestamp'].isin(prev_window_events['time_seconds'].unique().tolist() + [row['timestamp']]))
    ].shape

(69, 26)

In [220]:
first_frames_df['ball_ownership_changed'].value_counts()

ball_ownership_changed
False    136
True      58
Name: count, dtype: int64

In [97]:
time_window = pd.Timedelta(seconds=5)
row = first_frames_df.iloc[2]
ref_period_id = row['period_id']
ref_timestamp = row['timestamp']

start_time = ref_timestamp - time_window
end_time = ref_timestamp + time_window

filtered_event_df[
    (filtered_event_df['period_id'] == ref_period_id) &
    # (filtered_event_df['time_seconds'] >= start_time) &
    (filtered_event_df['time_seconds'] <= ref_timestamp)
    ]

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy,player,position,team,jID,pID,tID,xID,player_code
0,126315,0.0,0,1,0 days 00:00:02.160000,0 days 00:00:03.960000,4648,360389,62365.0,-1,-1,pass,4,Successful,4.0,-1.0,52.5840,35.8428,69.0480,46.8860,69.0480,46.8860,16.4640,11.0432,Kang Hyun Lee,CM,Away,24,360389,4648,6,A06
1,126315,5.0,2,1,0 days 00:00:04.640000,0 days 00:00:06,4648,62365,500133.0,-1,-1,pass,4,Successful,6.0,-1.0,68.9430,47.7428,69.8565,31.0964,69.8565,31.0964,0.9135,-16.6464,Youngkyu Ahn,CB,Away,6,62365,4648,3,A03
2,126315,NaN,4,1,0 days 00:00:06,NaT,4648,500133,NaN,-1,-1,dribble,2,NaN,NaN,NaN,69.8565,31.0964,57.4665,29.8928,NaN,NaN,-12.3900,-1.2036,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
3,126315,8.0,5,1,0 days 00:00:10.680000,0 days 00:00:12.440000,4648,500133,145701.0,-1,-1,pass,4,Successful,9.0,-1.0,57.4665,29.8928,59.6820,15.9664,59.6820,15.9664,2.2155,-13.9264,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
4,126315,9.0,7,1,0 days 00:00:12.440000,0 days 00:00:13.520000,4648,145701,77414.0,-1,-1,pass,4,Successful,11.0,-1.0,59.6820,15.9664,41.8845,14.3276,41.8845,14.3276,-17.7975,-1.6388,Mingi Lee,LB,Away,3,145701,4648,1,A01


In [ ]:
filtered_event_df[
    (filtered_event_df['period_id'] == ref_period_id) &
    (filtered_event_df['time_seconds'] >= ref_timestamp) &
    (filtered_event_df['time_seconds'] <= end_time)
    ]

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy,player,position,team,jID,pID,tID,xID,player_code
3,126315,8.0,5,1,0 days 00:00:10.680000,0 days 00:00:12.440000,4648,500133,145701.0,-1,-1,pass,4,Successful,9.0,-1.0,57.4665,29.8928,59.6820,15.9664,59.6820,15.9664,2.2155,-13.9264,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
4,126315,9.0,7,1,0 days 00:00:12.440000,0 days 00:00:13.520000,4648,145701,77414.0,-1,-1,pass,4,Successful,11.0,-1.0,59.6820,15.9664,41.8845,14.3276,41.8845,14.3276,-17.7975,-1.6388,Mingi Lee,LB,Away,3,145701,4648,1,A01
5,126315,NaN,9,1,0 days 00:00:13.520000,NaT,4648,77414,NaN,-1,-1,dribble,2,NaN,NaN,NaN,41.8845,14.3276,35.0595,14.4636,NaN,NaN,-6.8250,0.1360,Huigyun Lee,CF,Away,10,77414,4648,9,A09
6,126315,12.0,12,1,0 days 00:00:14.480000,NaT,4648,250102,-1.0,-1,-1,recovery,15,NaN,-1.0,13.0,35.0595,14.4636,35.0595,14.4636,NaN,NaN,0.0000,0.0000,Kunhee Lee,CF,Away,20,250102,4648,10,A10
7,126315,NaN,13,1,0 days 00:00:14.520000,NaT,4648,250102,NaN,-1,-1,dribble,2,NaN,NaN,NaN,35.0595,14.4636,38.9970,24.6500,NaN,NaN,3.9375,10.1864,Kunhee Lee,CF,Away,20,250102,4648,10,A10
